# Quantum Circuits & Gates — Detailed Notes (Session 2)
**Course:** CS490/5590 — Quantum Computing Applications in Data Science, AI, & Deep Learning  
**Instructor:** Luke Miller  
**Level:** Graduate / Advanced Undergraduate

> **Purpose of these notes.** A stand-alone reference for Session 2. It expands the slide bullets into definitions, short derivations, worked examples, Qiskit snippets, and mini-exercises (with brief solutions).

---

## Session Overview

**Objective.** Explore quantum circuits, single- and multi-qubit gates, and common visualization techniques (Bloch sphere and circuit diagrams). Connect these primitives to AI/DS use cases.

**Agenda.**
1. Recap of quantum basics  
2. What are quantum circuits?  
3. Single-qubit gates (Paulis, H, phase gates S/T, rotation gates)  
4. Multi-qubit gates (CNOT, CZ, controlled-U), basic decompositions  
5. Circuit composition & decomposition; depth/width  
6. Bloch sphere & circuit diagrams  
7. Applications in AI/DS  
8. Q&A

---

## 0) Recap from Session 1

- **Quantum vs classical.** Superposition, entanglement (multi-qubit), engineered interference; measurement collapses a state.  
- **Qubit.** $|\psi\rangle=\alpha|0\rangle+\beta|1\rangle$ with $|\alpha|^2+|\beta|^2=1$; amplitudes are complex.  
- **Born rule.** Measurement in $\{|0\rangle,|1\rangle\}$:$\Pr(0)=|\alpha|^2$, \$Pr(1)=|\beta|^2$.  
- **Qiskit.** You built/simulated a 1-qubit Hadamard circuit yielding ~50/50 counts.

**Today’s connection.** Gates are just unitary matrices acting on these vectors. Circuits are sequences of gates (plus measurement). We’ll learn to read, write, and reason about common gates and their compositions.

**Prompt for thought.** How does superposition enable possible ML advantages? (Short answer: it provides a high-dimensional feature space and the ability to shape interference patterns; the trick is extracting useful global properties with limited measurement.)

---

## 1) What are quantum circuits?

**Definition.** A *quantum circuit* is an ordered sequence of unitary gates applied to one or more qubits, optionally followed by measurements to produce classical outcomes.

**Components.**
- **Qubits.** Quantum registers holding $|\psi\rangle$ (possibly entangled across registers).
- **Gates.** Unitary operations $U$ with $U^\dagger U=I$.
- **Measurement.** Projective map from quantum to classical bits (irreversible).

**Mathematical view.** If a circuit applies gates $U_1, U_2, \dots, U_n$ to an initial state $|\psi_0\rangle$, then
$$
|\psi_{\text{out}}\rangle = U_n \cdots U_2 U_1 |\psi_0\rangle.
$$
**Composition is matrix multiplication**, applied right-to-left.

**AI tie-in.** Circuits can:
- **Encode data** as quantum states (feature maps/kernels).  
- **Implement cost-and-mixer layers** for variational optimization (e.g., QAOA).

**Minimal example.**
- Initialize $|0\rangle$, apply $H$, measure ⇒ ~50/50 counts.

---

## 2) Single-qubit gates

### 2.1 Pauli gates $X, Y, Z$
Matrices:
$$
X=\begin{pmatrix}0&1\\1&0\end{pmatrix},\quad
Y=\begin{pmatrix}0&-i\\i&0\end{pmatrix},\quad
Z=\begin{pmatrix}1&0\\0&-1\end{pmatrix}.
$$
Actions:
- $X|0\rangle=|1\rangle,\; X|1\rangle=|0\rangle$ (bit-flip).  
- $Y|0\rangle=i|1\rangle,\; Y|1\rangle=-i|0\rangle$ (bit-flip + phase).  
- $Z|0\rangle=|0\rangle,\; Z|1\rangle=-|1\rangle$ (phase-flip).

**Properties.**
- Hermitian and unitary (involutions): $X^2=Y^2=Z^2=I$.  
- Anti-commutation: $XZ=-ZX$, $XY=iZ$, $YZ=iX$, $ZX=iY$ up to phases.

**Bloch-sphere view.**  
Paulis are $\pi$-rotations:
$$
X=R_x(\pi),\quad Y=R_y(\pi),\quad Z=R_z(\pi),
$$
where $R_\alpha(\theta)=e^{-i\theta \sigma_\alpha/2}$.

**Application note.** Pauli expectation values $\langle X\rangle, \langle Y\rangle, \langle Z\rangle$ are the coordinates of a pure state on the Bloch sphere and are ubiquitous readouts in quantum ML and VQE.

---

### 2.2 Hadamard \(H\)
$$
H=\frac{1}{\sqrt{2}}\begin{pmatrix}1&1\\1&-1\end{pmatrix},\quad
H|0\rangle=|+\rangle=\frac{|0\rangle+|1\rangle}{\sqrt{2}},\quad
H|1\rangle=|-\rangle=\frac{|0\rangle-|1\rangle}{\sqrt{2}}.
$$
**Properties.** $H^2=I$. Conjugation identities: $HXH=Z,\; HZH=X$.

**Discussion.** Hadamard is *not* random: it deterministically sets amplitudes, which *later* gates can interfere with.

---

### 2.3 Phase gates $S,T$ and rotations
$$
S=\begin{pmatrix}1&0\\0&i\end{pmatrix},\\
T=\begin{pmatrix}1&0\\0&e^{i\pi/4}\end{pmatrix}.
$$
Relations: $S^2=Z,\; T^2=S,\; T^4=Z,\; T^8=I$.

**Rotations (continuous parameters).**
$$
R_x(\theta)=e^{-i\theta X/2},\quad
R_y(\theta)=e^{-i\theta Y/2},\quad
R_z(\theta)=e^{-i\theta Z/2}.
$$
Used heavily in variational circuits (fine-grained control of states).

**Universality (preview).** A common universal gate set is **Clifford+T**, e.g., $\{H, S, T, \text{CNOT}\}$.

---

## 3) Multi-qubit gates

### 3.1 Controlled-NOT (CNOT or CX)
Action on computational basis (control $q_c$, target $q_t$):
$$
\text{CNOT}|00\rangle=|00\rangle,\\
\text{CNOT}|01\rangle=|01\rangle,\\
\text{CNOT}|10\rangle=|11\rangle,\\
\text{CNOT}|11\rangle=|10\rangle.
$$
Matrix (ordering \(|00\rangle,|01\rangle,|10\rangle,|11\rangle\)):
$$
\text{CNOT}=
\begin{pmatrix}
1&0&0&0\\
0&1&0&0\\
0&0&0&1\\
0&0&1&0
\end{pmatrix}.
$$
**Creates entanglement.** $H$ on control then CNOT ⇒ Bell state $|\Phi^+\rangle=(|00\rangle+|11\rangle)/\sqrt{2}$.

**Qiskit endian note.** Qiskit uses little-endian ordering for statevectors: basis $|q_{n-1}\cdots q_0\rangle$. Bitstrings in counts are printed with most significant bit left; when in doubt, test tiny circuits to confirm mapping.

---

### 3.2 Controlled-Z (CZ) and controlled-\(U\)
- **CZ** multiplies $|11\rangle$ by $-1$ (diagonal $1,1,1,-1$).  
- **Identity:** $(I\otimes H)\,\text{CNOT}\,(I\otimes H)=\text{CZ}$.  
- **General controlled-$U$:** apply $U$ on target iff control is $|1\rangle$.

**Decomposition idea.** Many controlled gates reduce to a handful of CNOTs plus single-qubit rotations.

---

### 3.3 Useful decompositions/identities
- **SWAP** $=$ CNOT$_{1\to2}$ → CNOT$_{2\to1}$ → CNOT$_{1\to2}$.  
- **Basis change:** Conjugate by $H$ to flip X/Z roles (as above).  
- **Controlled rotations:** Often decomposed into CNOTs + half-angle single-qubit rotations.

---

## 4) Circuit composition & decomposition

**Composition.** Circuits are sequences of gates; tensors apply to disjoint qubits:
$$
(X\otimes I)\,(I\otimes H) = X\otimes H.
$$
**Decomposition.** Break complex gates into basic blocks to target a device’s native gate set (the transpiler will do this, but understanding it helps debugging and optimization).

**Depth & width.**
- **Width** = number of qubits used.  
- **Depth** = number of sequential layers (operations that cannot be parallelized).  
- In NISQ settings, *shallower is generally better* due to decoherence and gate error.

**Example: Bell circuit.** Depth ~2: $H$ on $q_0$, then CX($q_0\to q_1$).

---

## 5) Bloch sphere & circuit diagrams

**Bloch sphere.** Any single-qubit pure state corresponds to a point with spherical angles \((\theta,\phi)\):
$$
|\psi\rangle=\cos\frac{\theta}{2}|0\rangle + e^{i\phi}\sin\frac{\theta}{2}|1\rangle.
$$
- $Z$-axis: $|0\rangle$ (north), $|1\rangle$ (south).  
- Equator: Hadamard superpositions (e.g., $|\pm\rangle$).  
- Gate effects are rotations.

**Circuit diagrams.**
- Wires (qubits) run left→right through time.  
- Boxes are single-qubit gates; •—⊕ is CNOT (• control, ⊕ target).  
- In Qiskit: `qc.draw()` (`'text'` or `'mpl'`) to visualize.

---

## 6) Qiskit: hands-on snippets


### 6.1 Single-qubit identities (verify $HXH=Z$, $HZH=X$)
```python
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Operator
import numpy as np

sim = AerSimulator()

def unitary_of(circ):
    # Transpile to unitary-sim friendly form and extract matrix
    return Operator(circ).data

# HXH
qc_hxh = QuantumCircuit(1)
qc_hxh.h(0); qc_hxh.x(0); qc_hxh.h(0)
U_hxh = unitary_of(qc_hxh)

# Should equal Z (up to global phase)
Z = np.array([[1,0],[0,-1]], dtype=complex)
print("HXH ≈ Z ?", np.allclose(U_hxh, Z))

# HZH
qc_hzh = QuantumCircuit(1)
qc_hzh.h(0); qc_hzh.z(0); qc_hzh.h(0)
U_hzh = unitary_of(qc_hzh)
X = np.array([[0,1],[1,0]], dtype=complex)
print("HZH ≈ X ?", np.allclose(U_hzh, X))
```

### 6.2 Bell state & endianness sanity check
```python
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

qc = QuantumCircuit(2,2)
qc.h(0)
qc.cx(0,1)
qc.measure([0,1],[0,1])  # measure q0->c0, q1->c1
print(qc.draw('text'))

backend = AerSimulator()
counts = backend.run(transpile(qc, backend), shots=2000).result().get_counts()
print(counts)  # Expect ~{'00': 1000, '11': 1000}
```
> **Note on bitstrings.** With `measure([0,1],[0,1])`, the printed key `'ab'` corresponds to `c1c0` (msb left). For the Bell above, you’ll see mostly `'00'` and `'11'`.

### 6.3 Bloch visualization (single or multi-qubit)
```python
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector
import matplotlib.pyplot as plt

qc = QuantumCircuit(1)
qc.h(0); qc.s(0)      # put the qubit on equator then rotate around Z
psi = Statevector.from_label('0').evolve(qc)
plot_bloch_multivector(psi)  # shows Bloch coordinates
plt.show()
```

### 6.4 Decompositions: SWAP via three CNOTs
```python
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.cx(0,1)
qc.cx(1,0)
qc.cx(0,1)
print(qc.draw('text'))
```

### 6.5 Circuit depth/size metrics
```python
qc = QuantumCircuit(2)
qc.h(0); qc.cx(0,1); qc.z(1); qc.h(1)
print("Depth:", qc.depth(), "Size (#ops):", qc.size())
```

---

## 7) Applications in AI & Data Science (orientation)

- **Quantum kernels (QSVM).** Embed data $x\mapsto |\phi(x)\rangle$; compute kernel $K(x,x')=|\langle\phi(x)|\phi(x')\rangle|^2$. Choice of single-/multi-qubit gates shapes the feature space.
- **Variational models.** Parameterized rotations $R_\alpha(\theta)$ + entanglers (CNOT/CZ) form layers. Classical optimizers adjust $\theta$ to minimize a loss.
- **Optimization (QAOA).** Alternating $e^{-i\gamma C}$ (often built from CZs) with mixers $e^{-i\beta \sum X_i}$.

**Tiny feature map sketch (2-qubit).**
```python
from qiskit import QuantumCircuit

def feature_map(x0, x1):
    qc = QuantumCircuit(2)
    qc.ry(x0, 0)
    qc.ry(x1, 1)
    qc.cz(0, 1)
    qc.ry(x0, 0)   # repeat to enrich nonlinearity
    qc.ry(x1, 1)
    return qc
```
You’ll later compare overlaps of such states across samples as a kernel.

---

## 8) Worked algebra/examples

**Example A — Conjugation turns bit-flip into phase-flip.**  
Show $HXH=Z$. *Sketch:* Multiply matrices or use basis action:  
$H|0\rangle=|+\rangle,\; X|+\rangle=|+\rangle,\; H|+\rangle=|0\rangle$ and  
$H|1\rangle=|-\rangle,\; X|-\rangle=-|-\rangle,\; H|-\rangle=-|1\rangle$ ⇒ same as $Z$.

**Example B — CNOT + basis change = CZ.**  
$(I\otimes H)\, \text{CNOT}\, (I\otimes H)=\text{CZ}$. *Idea:* $H$ swaps $X \leftrightarrow Z$ on the target by conjugation, turning a target-$X$ into target-$Z$.

**Example C — SWAP via 3 CNOTs.**  
Act on basis states and verify outputs equal SWAP’s effect, or show matrices match.

---

## 9) Mini-exercises (brief solutions at end)

1. **Pauli composition.** Prove $XZ=-ZX$. (*Hint:* Multiply the 2×2 matrices.)  
2. **Rotation identity.** Verify $R_z(\pi)=Z$ and $R_x(\pi)=X$.  
3. **Hadamard conjugation.** Compute $HZH$ explicitly and confirm it equals $X$.  
4. **Bell construction.** Starting from $|00\rangle$, apply $H$ on qubit 0 then CNOT(0→1). Show the final state is $(|00\rangle+|11\rangle)/\sqrt{2}$.  
5. **CZ↔CNOT.** Use the identity $(I\otimes H)\, \text{CNOT}\, (I\otimes H)=\text{CZ}$ to implement CZ on hardware that only natively supports CNOT and single-qubit gates.  
6. **Depth vs error.** Explain in 3 sentences why lower circuit depth is typically better on NISQ devices.  
7. **Endianness check.** For a 2-qubit circuit that measures $|\Phi^+\rangle$, why do you see almost only `'00'` and `'11'` in Qiskit counts?

---

## 10) FAQ / common pitfalls

- **“My diagram shows CZ but hardware ran CNOTs.”** The transpiler decomposed CZ to native gates. That’s expected.  
- **“Why isn’t my Bloch vector exactly on the equator?”** Numerical rounding and small imaginary parts can shift plots slightly; normalize and check phases.  
- **“Counts aren’t exact.”** Finite sampling causes noise; increase `shots` to tighten estimates.  
- **“CNOT control/target confusion.”** Be explicit: `qc.cx(control, target)`. Swapping them changes the operation.  
- **“Statevector vs measurement.”** Once you measure, the simulator collapses the state. Inspect statevectors *before* measurement.

---

## 11) Summary (Session 2)

- Circuits = ordered products of unitaries; measure at the end to get classical data.  
- Single-qubit gates $X,Y,Z,H,S,T,R_\alpha(\theta)$ act as rotations on the Bloch sphere.  
- Multi-qubit gates (CNOT, CZ) enable entanglement; many gates interconvert via conjugations.  
- Composition/decomposition and depth/width are practical concerns for NISQ.  
- Visual tools (circuit drawers, Bloch plots) are essential for debugging and intuition.  
- These notions underpin quantum feature maps, kernels, and variational layers in AI/DS.

---

## 12) Looking ahead

- **Next Session:** Entanglement & Bell states (measurement statistics, partial trace), multi-qubit measurement, and simple tomography.  
- **Homework reminder (HW1):** Gate applications—verify identities in Qiskit, prepare a Bell state, and submit counts + a short explanation (include an endianness note).

---

## 13) Brief solutions to mini-exercises

1. $XZ=\begin{pmatrix}0&1\\1&0\end{pmatrix}\begin{pmatrix}1&0\\0&-1\end{pmatrix}
=\begin{pmatrix}0&-1\\1&0\end{pmatrix}=-\begin{pmatrix}0&1\\-1&0\end{pmatrix}=-ZX.$

2. $R_z(\pi)=e^{-i\pi Z/2}=\cos(\pi/2)I - i\sin(\pi/2)Z = -iZ \sim Z$ up to global phase; similarly $R_x(\pi)\sim X$.

3. Multiply $H Z H$ or use basis action to get $X$.

4. $|00\rangle \xrightarrow{H\otimes I} (|00\rangle+|10\rangle)/\sqrt{2} \xrightarrow{\text{CNOT}} (|00\rangle+|11\rangle)/\sqrt{2}.$

5. Implement CZ as `qc.h(target); qc.cx(control, target); qc.h(target)`.

6. Each gate introduces noise; deeper circuits accumulate error and are more likely to decohere before readout.

7. Because the Bell state yields perfectly correlated bits in the computational basis; Qiskit prints bitstrings with most significant classical bit on the left, so you see `'00'` and `'11'`.

